<a href="https://colab.research.google.com/github/YaCpotato/python/blob/master/GBDTCompareWithIris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import matplotlib as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix,classification_report
import time
from sklearn import model_selection
import pandas as pd

## データセットロード

In [28]:
iris = datasets.load_iris()
iris_df = pd.DataFrame(iris.data)
iris_df = iris_df.rename(columns={
    0: 'sepal_length',
    1: 'sepal_width',
    2: 'petal_length',
    3: 'petal_width'})
iris_df['target'] = iris.target
iris_df.head()

,sepal_length,sepal_width,petal_length,petal_width,target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [0]:
train_df, test_df = model_selection.train_test_split(iris_df, test_size=0.3) # 30%をテストに使う
train_df_y = train_df[['target']]
train_df_x = train_df.copy().drop('target', axis=1)
test_df_y = test_df[['target']]
test_df_x = test_df.copy().drop('target', axis=1)

In [0]:
import xgboost as xgb
clf = xgb.XGBClassifier()
clf_cv = model_selection.GridSearchCV(clf, {'max_depth': [2,4,6], 'n_estimators': [50,100,200]}, verbose=1)

In [31]:
%%time
clf_cv.fit(train_df_x, [i[0] for i in train_df_y.values])
clf = xgb.XGBClassifier(**clf_cv.best_params_)
clf.fit(train_df_x, [i[0] for i in train_df_y.values])

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


CPU times: user 997 ms, sys: 64.4 ms, total: 1.06 s
Wall time: 1.07 s


[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:    1.0s finished


In [32]:
print(clf_cv.best_params_, clf_cv.best_score_)

{'max_depth': 4, 'n_estimators': 50} 0.9523809523809523


In [33]:
pred = clf.predict(test_df_x)
print(confusion_matrix([i[0] for i in test_df_y.values], pred))
print(classification_report([i[0] for i in test_df_y.values], pred))

[[11  0  0]
 [ 0 14  1]
 [ 0  1 18]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        11
           1       0.93      0.93      0.93        15
           2       0.95      0.95      0.95        19

    accuracy                           0.96        45
   macro avg       0.96      0.96      0.96        45
weighted avg       0.96      0.96      0.96        45



In [0]:
import lightgbm as lgb
params = {
    'n_estimators': 10000,  # 大きめにとっておく
}

lgb = lgb.LGBMClassifier(**params)

In [35]:
%%time
lgb.fit(train_df_x, [i[0] for i in train_df_y.values],verbose=1)

CPU times: user 1.18 s, sys: 143 ms, total: 1.32 s
Wall time: 1.33 s


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=10000, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [36]:
pred = clf.predict(test_df_x)
print(confusion_matrix([i[0] for i in test_df_y.values], pred))
print(classification_report([i[0] for i in test_df_y.values], pred))

[[11  0  0]
 [ 0 14  1]
 [ 0  1 18]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        11
           1       0.93      0.93      0.93        15
           2       0.95      0.95      0.95        19

    accuracy                           0.96        45
   macro avg       0.96      0.96      0.96        45
weighted avg       0.96      0.96      0.96        45



In [37]:
!pip install catboost

In [0]:
from catboost import CatBoostClassifier
from catboost import Pool

train_pool = Pool(train_df_x, label=[i[0] for i in train_df_y.values])
test_pool = Pool(test_df_x, label=[i[0] for i in test_df_y.values])

In [0]:
cb = CatBoostClassifier(iterations=1000, loss_function="MultiClass",classes_count=3)

In [40]:
%%time
cb.fit(train_pool)

0:	learn: 1.0633521	total: 1.44ms	remaining: 1.44s
1:	learn: 1.0307501	total: 2.26ms	remaining: 1.13s
2:	learn: 1.0036780	total: 2.89ms	remaining: 961ms
3:	learn: 0.9751172	total: 3.54ms	remaining: 882ms
4:	learn: 0.9462098	total: 6.23ms	remaining: 1.24s
5:	learn: 0.9178737	total: 6.87ms	remaining: 1.14s
6:	learn: 0.8980286	total: 7.14ms	remaining: 1.01s
7:	learn: 0.8748967	total: 7.79ms	remaining: 966ms
8:	learn: 0.8532728	total: 8.44ms	remaining: 929ms
9:	learn: 0.8331947	total: 9.16ms	remaining: 907ms
10:	learn: 0.8129926	total: 9.83ms	remaining: 884ms
11:	learn: 0.7911762	total: 10.5ms	remaining: 863ms
12:	learn: 0.7711453	total: 11.1ms	remaining: 844ms
13:	learn: 0.7490227	total: 11.8ms	remaining: 829ms
14:	learn: 0.7319817	total: 12.4ms	remaining: 814ms
15:	learn: 0.7135330	total: 13ms	remaining: 802ms
16:	learn: 0.6982215	total: 13.6ms	remaining: 787ms
17:	learn: 0.6796852	total: 14.2ms	remaining: 777ms
18:	learn: 0.6660435	total: 14.9ms	remaining: 767ms
19:	learn: 0.6512586	tot

In [41]:
# テストデータを予測する
Y_pred = cb.predict(test_pool, prediction_type='Class')
Y_test = test_df_y.values
# 精度 (Accuracy) を計算する
from sklearn.metrics import confusion_matrix,classification_report
print (confusion_matrix(Y_test, Y_pred))
print (classification_report(Y_test, Y_pred))

[[11  0  0]
 [ 0 14  1]
 [ 0  1 18]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        11
           1       0.93      0.93      0.93        15
           2       0.95      0.95      0.95        19

    accuracy                           0.96        45
   macro avg       0.96      0.96      0.96        45
weighted avg       0.96      0.96      0.96        45

